In [9]:
from transformers import (
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
)

from datasets import load_dataset, Dataset
from huggingface_hub import login
import pandas as pd
import numpy as np
import evaluate
import shutil
import torch
import wandb
from huggingface_hub import login
from dotenv import load_dotenv
import os
import wandb
load_dotenv()

True

In [10]:
wandb.login(key=os.getenv("WANDB_API_KEY"))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/mac/.netrc


True

In [11]:
login(os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [12]:
dataset = load_dataset("ehovy/race", "all")

In [13]:
MODEL = 't5-small'                  # the Model name here
BATCH_SIZE = 8                      # The batch size here
NUM_PROCS = 10                      # The num of proccess here
EPOCHS = 3                          # The number of training Epochs here
OUT_DIR = 'results_t5small'         # the outout directory name here
MAX_LENGTH = 512                    # the max length of the sequence here

In [14]:
print(f"Dataset type: {type(dataset)}")
print(f"Dataset length: {len(dataset)}")
print(f"Dataset keys: {dataset.keys()}")

Dataset type: <class 'datasets.dataset_dict.DatasetDict'>
Dataset length: 3
Dataset keys: dict_keys(['test', 'train', 'validation'])


In [15]:
train_dataset = dataset["train"]
eval_dataset = dataset['validation']
test_dataset = dataset['test']

In [16]:
print(f"Train section type: {type(train_dataset)}")
print(f"Train section length: {len(train_dataset)}")

Train section type: <class 'datasets.arrow_dataset.Dataset'>
Train section length: 87866


In [ ]:
print(train_dataset[0].keys())

dict_keys(['example_id', 'article', 'answer', 'question', 'options'])


In [18]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)


tokenizer = AutoTokenizer.from_pretrained(MODEL)
MAX_LENGTH = 512


def preprocess_function(examples):
    inputs = []
    targets = []

    for article, question, options, answer in zip(
        examples["article"], examples["question"], examples["options"], examples["answer"]
    ):
        correct_index = ord(answer) - ord("A")
        correct_answer = options[correct_index]
        distractors = [opt for i, opt in enumerate(
            options) if i != correct_index]

        for distractor in distractors:
            prompt = f"generate distractor: {question} answer: {correct_answer} context: {article}"
            inputs.append(prompt)
            targets.append(distractor)

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    label_tokens = tokenizer(
        text_target=targets,
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = label_tokens["input_ids"]

    return model_inputs

In [19]:
tokenized_train_dataset = train_dataset.map(
    preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_eval_dataset = eval_dataset.map(
    preprocess_function, batched=True, remove_columns=eval_dataset.column_names)

In [20]:
tokenized_train_dataset[0]

{'input_ids': [3806,
  15980,
  127,
  10,
  101,
  54,
  214,
  45,
  8,
  5454,
  24,
  8,
  2291,
  930,
  38,
  3,
  9,
  834,
  5,
  1525,
  10,
  3145,
  2625,
  10,
  2506,
  471,
  27,
  6812,
  28,
  128,
  13,
  82,
  481,
  81,
  125,
  79,
  1114,
  12,
  103,
  227,
  79,
  9461,
  6,
  11,
  125,
  773,
  13,
  613,
  11136,
  79,
  816,
  79,
  141,
  5,
  9246,
  24,
  27,
  3884,
  481,
  113,
  33,
  761,
  12,
  36,
  6659,
  6,
  27,
  47,
  5597,
  103,
  253,
  24,
  167,
  816,
  24,
  79,
  133,
  59,
  36,
  3,
  179,
  12,
  129,
  8,
  2476,
  79,
  1114,
  406,
  96,
  670,
  1583,
  199,
  1280,
  96,
  5680,
  773,
  13,
  199,
  19,
  24,
  4609,
  27,
  1380,
  6,
  11873,
  135,
  12,
  817,
  140,
  24,
  79,
  133,
  174,
  3,
  9,
  42,
  384,
  1565,
  12,
  199,
  135,
  91,
  5,
  96,
  134,
  450,
  1304,
  63,
  3,
  976,
  80,
  18606,
  5,
  27,
  47,
  1134,
  491,
  8715,
  57,
  24,
  1773,
  5,
  94,
  1330,
  24,
  8,
  13819,
  13,
  469

In [21]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")

total_trainable_params = sum(p.numel()
                             for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

60,506,624 total parameters.
60,506,624 training parameters.


In [23]:
rouge = evaluate.load("rouge")

In [24]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(
        predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds,
                           references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(
        pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [25]:
training_args = Seq2SeqTrainingArguments(
    output_dir=OUT_DIR,               # Directory where the model outputs will be saved
    num_train_epochs=EPOCHS,          # Number of training epochs
    per_device_train_batch_size=BATCH_SIZE,  # Batch size for training
    per_device_eval_batch_size=BATCH_SIZE,   # Batch size for evaluation
    warmup_steps=500,                 # Number of steps for learning rate warmup
    weight_decay=0.01,                # Weight decay for regularization
    logging_dir=OUT_DIR,              # Directory for logging during training
    logging_steps=10,                 # How often to log training progress
    # Evaluation strategy (e.g., 'steps' or 'epoch')
    eval_strategy='steps',
    # Frequency of evaluation during training (in steps)
    eval_steps=1000,
    save_strategy='epoch',            # Save the model at the end of each epoch
    save_steps=1000,                  # Save the model every 1000 steps
    save_total_limit=3,               # Keep only the 3 most recent models to save space
    learning_rate=5e-4,               # Learning rate for optimization
    # Number of workers for loading data (for parallelization)
    dataloader_num_workers=4,
    # Report training progress to W&B (Weights and Biases)
    report_to='wandb',
    predict_with_generate=True,       # Use model's `generate` method for predictions
    push_to_hub=True,                 # Push the model to the Hugging Face Model Hub
    remove_unused_columns=False
)

In [26]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/var/folders/k5/1hnrvwx15ps3tnfdyvmjz25h0000gn/T/ipykernel_4308/3142021100.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [27]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


/Users/mac/Projects/fine-tuning/T5-distractor-generation/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.push_to_hub(repo_id="henryhoangduong/T5-small-distractor-generation")